# Web Scraping - Strava

In [1]:
## Biblioteca necessárias

import pandas as pd
from time import sleep
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException


## Iniciando uma instância do google chrome

In [25]:
# Pega o caminho do executável do ChromeDriver
service = Service(ChromeDriverManager().install())  # Corrigido, agora está criando o objeto Service corretamente.

# Usado para definir as preferências para o navegador Chrome
options = webdriver.ChromeOptions()

# Inicia-se uma instância do Google Chrome com as definições
driver = webdriver.Chrome(service=service, options=options)  # Agora a passagem dos parâmetros está correta.

sleep(1)

### Entrando no Site - Strava
url = 'https://www.strava.com/segments'
url_2 = 'https://www.strava.com/segments/35231066' # Segmento lamision35km2023
driver.get(url)

#Login 
usuario="joaomarcos1296@gmail.com"
senha="Jo@o1296"


campo_usuario = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(('xpath','//*[@id="mobile-email"]')))
campo_usuario.send_keys(usuario)

campo_senha = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(('xpath','//*[@id="__next"]/div/div[2]/div[1]/div[1]/div/form/div[1]/div[2]/div/input')))
campo_senha.send_keys(senha)

botao_login = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(('xpath','//*[@id="__next"]/div/div[2]/div[1]/div[1]/div/form/div[2]/button'))) 
botao_login.click()
sleep(5)


# Acessa a Url Alvo
driver.get(url_2)
sleep(3)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x007702D3+25139]
	(No symbol) [0x006FB284]
	(No symbol) [0x005D04A3]
	(No symbol) [0x00618338]
	(No symbol) [0x0061868B]
	(No symbol) [0x00660F62]
	(No symbol) [0x0063CF84]
	(No symbol) [0x0065E6E1]
	(No symbol) [0x0063CD36]
	(No symbol) [0x0060BD29]
	(No symbol) [0x0060D064]
	GetHandleVerifier [0x00A7B193+3215603]
	GetHandleVerifier [0x00A9230A+3310186]
	GetHandleVerifier [0x00A8C522+3286146]
	GetHandleVerifier [0x00809CD0+654384]
	(No symbol) [0x0070460D]
	(No symbol) [0x007014F8]
	(No symbol) [0x00701697]
	(No symbol) [0x006F3D70]
	BaseThreadInitThunk [0x76845D49+25]
	RtlInitializeExceptionChain [0x76FECDEB+107]
	RtlGetAppContainerNamedObjectPath [0x76FECD71+561]


In [32]:
def verifica_nomes_atletas():
    try:
        # Tente encontrar um elemento que contenha os nomes dos atletas
        driver.find_element(By.XPATH, '//*[@id="results"]/table/tbody/tr/td[2]/a')
        return True  # Se encontrar, retorna True (há nomes de atletas)
    except NoSuchElementException:
        return False 

def verifica_num_pag():
    try:
        num_paginas = len(driver.find_elements(By.XPATH, '//*[@id="results"]/nav/ul/li'))
        if num_paginas == 0:
            num_paginas = 1
        else:
            num_paginas
    except:
        num_paginas = 1
    
    return num_paginas

def clicar_na_pagina(i):
        try:
            driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/nav/ul/li[{i+1}]/a',
            ).click()
            sleep(1)
        except Exception:
            pass

def obter_nome_atleta(i, j):
        try:
            if i == 1 and j == 1:
                return driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/strong/a',
                ).text
            else:
                return driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/a',
                ).text
        except Exception:
            return None


def pega_nome_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 3
    num_pag = verifica_num_pag()
    nomes_atletas = []

    # Ajustar o número de páginas
    if num_pag == 1:
        num_pag += 1
    else:
        num_pag -= 1

    for i in range(1, num_pag):
        clicar_na_pagina(i)
        for j in range(s, n):
            nome = obter_nome_atleta(i, j)
            if nome:
                nomes_atletas.append(nome)
            else:
                break

    return nomes_atletas

In [ ]:
def pega_dados_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 26
    num_pag = verifica_num_pag()

    for i in range(1, num_pag-1):
        clicar_na_pagina(i)
        for j in range(s, n):
            try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta
                driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/nav/ul/li[{i+1}]/a',
                    ).click()
                sleep(2)
                driver.find_element(
                    by = By.XPATH,
                    value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                    ).click() # Clicando no link do atleta
                driver.back()
                sleep(2)
        
            except Exception: # Caso ja esteja na aba correta, vai direto pro atleta
                try:
                    driver.find_element(
                        by = By.XPATH,
                        value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                        ).click() # Clicando no link do atleta
                    driver.back()
                    sleep(2)
                except Exception:
                    break

In [5]:
pega_dados_atletas()

In [34]:
######## Pega dados atçeta 2.0
def tem_tabela_voltas():
    """
    Verifica se a aba "Voltas" está disponível no menu.
    """
    try:
        elemento = driver.find_element(By.XPATH, '//*[@id="pagenav"]/li[3]/a')
        return "Voltas" in elemento.text
    except Exception:
        return False

def pega_dados_atletas():
    """
    Percorre a lista de atletas, entra na página de cada um, coleta seu nome e extrai a tabela de voltas ou a tabela inicial.
    """
    s = 1
    n = 3
    num_pag = verifica_num_pag()
    nomes_atletas = pega_nome_atletas()
    
    dados_coletados = []
    
    for i in range(1, num_pag - 1):
        clicar_na_pagina(i)
        for j in range(s, n):
            try:
                atleta_link = driver.find_element(By.XPATH, f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a')
                atleta_nome = nomes_atletas.pop(0) if nomes_atletas else f"Atleta_{i}_{j}"
                atleta_link.click()
                sleep(2)
                
                # Verifica se existe a aba "Voltas"
                if tem_tabela_voltas():
                    try:
                        driver.find_element(By.XPATH, '//*[@id="pagenav"]/li[3]/a').click() 
                        sleep(2)
                        df = pega_tabela_voltas()
                    except: 
                        driver.find_element(By.XPATH, '//*[@id="pagenav"]/li[4]').click() 
                        sleep(2)
                        df = pega_tabela_voltas()
                else:
                    df = pega_tabela_inicial()
                
                # Adiciona o nome do atleta à tabela
                df.insert(0, "Nome Atleta", atleta_nome)
                dados_coletados.append(df)
                
                driver.back()
                sleep(2)
                
            except Exception as e:
                print(f"Erro ao acessar atleta {j} na página {i}: {e}")
                break
    
    # Concatenar todos os DataFrames
    if dados_coletados:
        df_final = pd.concat(dados_coletados, ignore_index=True)
        df_final.to_csv("dados_atletas.csv", index=False)
        print("Dados salvos em 'dados_atletas.csv'")
    else:
        print("Nenhum dado coletado.")
    
    return df_final if dados_coletados else None



In [35]:
verifica_num_pag()

7

In [36]:
df = pega_dados_atletas()

Erro ao acessar atleta 1 na página 4: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="run-efforts-table"]/section/table"}
  (Session info: chrome=133.0.6943.98); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007702D3+25139]
	(No symbol) [0x006FB284]
	(No symbol) [0x005D04A3]
	(No symbol) [0x00618338]
	(No symbol) [0x0061868B]
	(No symbol) [0x00660F62]
	(No symbol) [0x0063CF84]
	(No symbol) [0x0065E6E1]
	(No symbol) [0x0063CD36]
	(No symbol) [0x0060BD29]
	(No symbol) [0x0060D064]
	GetHandleVerifier [0x00A7B193+3215603]
	GetHandleVerifier [0x00A9230A+3310186]
	GetHandleVerifier [0x00A8C522+3286146]
	GetHandleVerifier [0x00809CD0+654384]
	(No symbol) [0x0070460D]
	(No symbol) [0x007014F8]
	(No symbol) [0x00701697]
	(No symbol) [0x006F3D70]
	BaseThreadInitThunk [0x76845D49+25]
	RtlInitializeExceptionChain [0x76FECDEB

In [76]:
def pega_sexo_nomes(df):
    driver.refresh()
    for i in range(2, 4):
        # Clica no expandir os sexos
        driver.find_element(
            by=By.XPATH,
            value='//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/button'
        ).click()
        sleep(2)

        # Clica no sexo
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a'
        ).click()
        
        # Pega o texto do sexo
        sexo_texto = driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a'
        ).text

        sleep(3)
        
        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame({'sexo': [sexo_texto] * len(nomes_atletas), 'nome_atleta': nomes_atletas})
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)
    
    return  df 

In [77]:
# Exemplo de uso
df_sexo = pd.DataFrame(columns=['sexo', 'nome_atleta'])
df_sexo = pega_sexo_nomes(df_sexo)
df_sexo

except
except


,sexo,nome_atleta
0,Homens,André Mapa
1,Homens,Alexandre Santiago
2,Homens,Caiê Alonso
3,Homens,Mikael Martins
4,Homens,Victor Caetano
...,...,...
93,Mulheres,Luiza Ferreira Luvizotto
94,Mulheres,Lillian Oliveira
95,Mulheres,Rebeka Oppitz
96,Mulheres,Anamaria Bacci


In [31]:
nomes_corredores = pega_nome_atletas()

In [32]:
len(nomes_corredores)

25

In [33]:
print(nomes_corredores)

['Caiê Alonso', 'Mikael Martins', 'Gabriel Moreira', 'Caique Gabriel', 'Arnaldo Junio Rocha Pinho', 'Ferdinanda Alcântara', 'Guilherme FitHOUSE', 'Odilon da Silva Júnior', 'Davi Sathler', 'Cristian Maciel (Atleta HRunning)', 'Dielice Viegas', 'Christine Spilka', 'Leonardo de Magalhães', 'Raphael Castro', 'Raone Faria', 'Mayara Miotto', 'Thomaz Carvalho', 'Flávio Rodrigues', 'Lucas Goldin', 'Rafael Gregório', 'Sávio Antoniel | @savioantonielc', 'Lucas Diniz', 'Romildo Junior', 'Tamiris Barbosa 🌻', 'Maria Lúcia Simois']


In [ ]:
nomes_atletas = []
pega_nomes_atletas(nomes_atletas)

In [55]:
def pega_faixas_nomes(df):
    driver.refresh()
    for i in range(2, 11):
        # Clica no expandir as idades
        driver.find_element(
            by=By.XPATH,
            value='//*[@id="premium-enhanced"]/ul/ul[2]/li[11]/button'
        ).click()
        sleep(2)

        # Clica na faixa idade
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).click()
        
        # Pega o texto da faixa etaria
        faixa_texto = driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).text

        sleep(3)
        
        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame({'faixa_etaria': [faixa_texto] * len(nomes_atletas), 'nome_atleta': nomes_atletas})
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

In [57]:
# Exemplo de uso
df_faixas_idades = pd.DataFrame(columns=['faixa_etaria', 'nome_atleta'])
df_faixas_idades = pega_faixas_nomes(df_faixas_idades)
df_faixas_idades

,faixa_etaria,nome_atleta
0,20 a 24,Victor Caetano
1,20 a 24,Breno Cordeiro
2,20 a 24,Lucas Serra
3,20 a 24,Alex Junior Aj
4,25 a 34,Caiê Alonso
...,...,...
99,45 a 54,Viviane Montevequi
100,45 a 54,Ana Cristina
101,55 a 64,Pretinha Miguel
102,55 a 64,Magali Bellam


In [62]:
def pega_pesos_nomes(df):
    driver.refresh()
    for i in range(2, 10):
        # Clica no expandir os pesos
        driver.find_element(
            by=By.XPATH,
            value='//*[@id="premium-enhanced"]/ul/ul[3]/li[10]/button' 
        ).click()
        sleep(2)

        # Clica na faixa peso
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).click()
        
        # Pega o texto da faixa etaria
        peso_texto = driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).text

        sleep(3)
        
        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame({'peso': [peso_texto] * len(nomes_atletas), 'nome_atleta': nomes_atletas})
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

In [63]:
# Exemplo de uso
df_pesos = pd.DataFrame(columns=['peso', 'nome_atleta'])
df_pesos = pega_pesos_nomes(df_pesos)
df_pesos

except
except
except
except
except
except


,faixa_etaria,nome_atleta,peso
0,NaN,Breno Cordeiro,54 kg e abaixo
1,NaN,Ligia Almeida Fechio,54 kg e abaixo
2,NaN,Ferdinanda Alcântara,54 kg e abaixo
3,NaN,Chris 🏃🏼‍♀️🚴🏼‍♀️,54 kg e abaixo
4,NaN,Graci Lindenmayr #WRTrail,54 kg e abaixo
...,...,...,...
88,NaN,Danilo Patricio Barbosa,85 a 94 kg
89,NaN,Ana Cristina,85 a 94 kg
90,NaN,Flávio Rodrigues,95 kg a 104 kg
91,NaN,Tony Ortiz,95 kg a 104 kg


## Tabelas dados voltas corredores

In [ ]:
def pega_tabela_voltas():
    """
    Extrai a tabela de voltas da página de atividades do Strava e salva como CSV.
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="efforts-table"]') 

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")] 
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


# Exemplo de uso:
url_strava = "https://www.strava.com/activities/9680684368/laps"
df_voltas = pega_tabela_voltas()
print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

  Volta Distância Tempo     Ritmo Elev.       FC
0     1   1,00 km  4:45  4:45 /km  -1 m  152 bpm
1     2   1,00 km  4:58  4:58 /km  85 m  157 bpm
2     3   1,00 km  4:55  4:55 /km  59 m  163 bpm
3     4   1,00 km  6:03  6:03 /km  94 m  175 bpm
4     5   1,00 km  5:57  5:57 /km   0 m  170 bpm


In [16]:
df_voltas.head()

,Volta,Distância,Tempo,Ritmo,Elev.,FC
0,1,"1,00 km",4:45,4:45 /km,-1 m,152 bpm
1,2,"1,00 km",4:58,4:58 /km,85 m,157 bpm
2,3,"1,00 km",4:55,4:55 /km,59 m,163 bpm
3,4,"1,00 km",6:03,6:03 /km,94 m,175 bpm
4,5,"1,00 km",5:57,5:57 /km,0 m,170 bpm


In [17]:
df_voltas.tail()

,Volta,Distância,Tempo,Ritmo,Elev.,FC
30,31,"1,00 km",9:47,9:47 /km,146 m,138 bpm
31,32,"1,00 km",5:35,5:35 /km,-66 m,156 bpm
32,33,"1,00 km",5:23,5:23 /km,-99 m,155 bpm
33,34,"1,00 km",4:25,4:25 /km,-26 m,155 bpm
34,35,"0,24 km",1:05,4:31 /km,3 m,158 bpm


## Pega tabela de dados pagina inicial

In [ ]:
def pega_tabela_inicial():
    """
    Extrai a tabela de voltas da página inicial do segmento do atleta do Strava
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="run-efforts-table"]/section/table')

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")] 
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


# Exemplo de uso:
url_strava = "https://www.strava.com/activities/9681673457#3128470765882939836"
df_voltas = pega_tabela_inicial()
print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

  KM     Ritmo       RAI Elev.
0  1  4:43 /km  4:40 /km  -1 m
1  2  5:07 /km  3:41 /km  84 m
2  3  5:14 /km  4:12 /km  61 m
3  4  6:30 /km  4:14 /km  94 m
4  5  6:38 /km  4:46 /km   0 m


In [19]:
df_voltas.head()

,KM,Ritmo,RAI,Elev.
0,1,4:43 /km,4:40 /km,-1 m
1,2,5:07 /km,3:41 /km,84 m
2,3,5:14 /km,4:12 /km,61 m
3,4,6:30 /km,4:14 /km,94 m
4,5,6:38 /km,4:46 /km,0 m


In [20]:
df_voltas.tail()

,KM,Ritmo,RAI,Elev.
29,30,6:26 /km,5:29 /km,46 m
30,31,10:05 /km,5:34 /km,115 m
31,32,5:26 /km,4:38 /km,-54 m
32,33,4:59 /km,5:05 /km,-120 m
33,34,4:31 /km,4:29 /km,-1 m


### Segments

In [ ]:

## Procura Segmentos Alvo
segmento = "La Mision"

campo_seg = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="keywords"]')))
campo_seg.send_keys(segmento)

botao_run = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="activity-type"]/ul/li[2]/a'))) 
botao_run.click()

botao_busca = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="climb-search-button"]'))) 
botao_busca.click()


